::: {.callout-note}

We are using Spatial Functions and the GEOMETRY data type below, and as of 2025-07-16, you need to use Databricks Runtime 17.1+ for this. It is expected that this feature becomes available on Serverless Compute and DBSQL as well soon, see [here](https://www.linkedin.com/feed/update/urn:li:activity:7351276934338641920/).

:::

In [ ]:
%pip install duckdb lonboard shapely --quiet
import duckdb
from lonboard import viz

In [ ]:
HTTP_BLOCKED = True

if HTTP_BLOCKED:
    import os
    from urllib.parse import urlparse

    import requests

    ARCHITECTURE = "linux_amd64"
    duckdb_version = duckdb.__version__
    url = f"https://extensions.duckdb.org/v{duckdb_version}/{ARCHITECTURE}/httpfs.duckdb_extension.gz"

    output_file = os.path.basename(urlparse(url).path)
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(output_file, "wb") as f:
        f.write(response.content)

    duckdb.install_extension(output_file)

    os.remove(output_file)

    duckdb.sql("SET custom_extension_repository='https://extensions.duckdb.org'")

In [ ]:
duckdb.sql("install spatial; load spatial")

In [ ]:
%sql
create
or replace table tmp_geometries as
select
  "Onze Lieve Vrouwetoren" as name,
  st_transform(st_point(155000, 463000, 28992), 4326) as geometry
union all
select
  "Bermuda Triangle" as name,
  st_makepolygon(
    st_makeline(
      array(
        st_point(- 80.1935973, 25.7741566, 4326),
        st_point(- 64.7563086, 32.3040273, 4326),
        st_point(- 66.1166669, 18.4653003, 4326),
        st_point(- 80.1935973, 25.7741566, 4326)
      )
    )
  ) as geometry;

select
  *
from
  tmp_geometries

In [ ]:
dfa = spark.table("tmp_geometries").limit(10_000).toArrow()

(In the below example, if the SRID is 4326, then the `st_transform` is redundant.)

In [ ]:
query = duckdb.sql("""
select
    * replace(st_transform(
        st_geomfromwkb(geometry.wkb),
        concat('EPSG:', geometry.srid),
        'OGC:CRS84'
    ) as geometry)
from dfa""")
query

In [ ]:
viz(query).as_html()